# Initial Mess Around

Want to make a three true outcomes leader board, with maybe a would it dong? cross over

In [1]:
from pybaseball import statcast
from pybaseball import playerid_reverse_lookup
from pybaseball import statcast_batter, spraychart

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# this defaults to yesterday
end_dt = '2023-04-05'

df = statcast(start_dt=end_dt, end_dt=end_dt)

print(df.shape)

This is a large query, it may take a moment to complete


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.51it/s]

(3520, 92)


In [3]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1826,SL,2023-04-05,78.2,-3.27,4.05,"Thompson, Ryan",657041,657044,field_out,hit_into_play,...,7,7,2,2,7,Standard,Standard,77,-0.004,-0.349
1871,SI,2023-04-05,92.3,-3.51,4.18,"Thompson, Ryan",657041,657044,NaN,foul,...,7,7,2,2,7,Standard,Standard,268,0.0,-0.137
1918,SI,2023-04-05,92.0,-3.35,4.08,"Thompson, Ryan",657041,657044,NaN,called_strike,...,7,7,2,2,7,Standard,Standard,266,0.0,-0.09
2001,SL,2023-04-05,79.2,-3.47,4.03,"Thompson, Ryan",657041,657044,NaN,ball,...,7,7,2,2,7,Standard,Standard,73,0.0,0.08
2046,SI,2023-04-05,91.8,-3.43,4.05,"Thompson, Ryan",682928,657044,field_out,hit_into_play,...,7,7,2,2,7,Standard,Standard,270,-0.011,-0.449


In [40]:
df.home_team.value_counts()

NYM    312
CHC    300
WSH    298
BOS    274
SEA    273
TOR    271
COL    265
DET    263
NYY    246
Name: home_team, dtype: int64

In [41]:
df.events.value_counts()

field_out                    289
strikeout                    144
single                        98
walk                          41
double                        24
force_out                     18
home_run                      15
field_error                    8
grounded_into_double_play      8
hit_by_pitch                   6
triple                         6
sac_fly                        6
fielders_choice                2
sac_bunt                       1
double_play                    1
Name: events, dtype: int64

In [4]:
tto_events = df.loc[df["events"].isin(["strikeout", "walk", "home_run"])]

In [5]:
tto_events.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2194,SI,2023-04-05,90.2,-3.44,4.03,"Thompson, Ryan",645302,657044,walk,ball,...,7,7,2,2,7,Standard,Standard,264,0.009,0.34
2629,SL,2023-04-05,77.3,-3.69,4.1,"Thompson, Ryan",656448,657044,walk,ball,...,7,7,2,2,7,Standard,Standard,76,0.004,0.237
1271,SI,2023-04-05,93.8,-1.25,5.56,"Ward, Thaddeus",642350,663658,strikeout,called_strike,...,2,7,2,7,2,Standard,Standard,227,0.0,-0.157
2043,ST,2023-04-05,81.4,-1.42,5.72,"Ward, Thaddeus",670623,663658,strikeout,swinging_strike_blocked,...,2,6,2,6,2,Standard,Standard,56,0.008,-0.408
2234,SI,2023-04-05,93.5,-1.43,5.48,"Ward, Thaddeus",677551,663658,walk,ball,...,2,6,2,6,2,Standard,Standard,232,-0.007,0.34


In [6]:
# ALSO NEED TO GET THE TOTAL OF PA FOR THE GAME!

outcome_counts = tto_events.groupby("batter")["events"].value_counts().unstack(fill_value=0)



In [7]:
outcome_counts.head()

events,home_run,strikeout,walk
batter,,,
456781,0,1,0
457759,0,0,1
467793,1,1,1
474832,0,3,0
502671,0,0,2


In [8]:
three_outcomes = outcome_counts.loc[(outcome_counts["home_run"] > 0) & (outcome_counts["strikeout"] > 0) & (outcome_counts["walk"] > 0)]

# Now try to calculate only three true outcomes

total_counts = df.groupby("batter")["events"].count()

three_outcomes = three_outcomes.merge(total_counts, on='batter', how='left')

three_outcomes

,home_run,strikeout,walk,events
batter,,,,
467793,1,1,1,5
676116,1,2,1,4


In [9]:
only_three_outcomes = three_outcomes.loc[three_outcomes[["home_run", "strikeout", "walk"]].sum(axis=1) == three_outcomes["events"]]

In [10]:
only_three_outcomes

,home_run,strikeout,walk,events
batter,,,,
676116,1,2,1,4


In [11]:
player_ids = only_three_outcomes.index

player_names = playerid_reverse_lookup(player_ids, key_type='mlbam')

Gathering player lookup table. This may take a moment.


In [12]:
only_three_outcomes = only_three_outcomes.merge(player_names[["name_first", "name_last", "key_mlbam"]],
                                                left_on='batter',
                                                right_on='key_mlbam',
                                                how="left")

In [13]:
only_three_outcomes

,home_run,strikeout,walk,events,name_first,name_last,key_mlbam
0,1,2,1,4,ryan,noda,676116


In [51]:
only_three_outcomes["run_date"] = end_dt
only_three_outcomes["has_been_posted"] = False
only_three_outcomes["posted_time"] = None


In [52]:
#only_three_outcomes.to_csv("data/tto.csv", mode="w", index = False) 

In [53]:
last_run = pd.read_csv("data/tto.csv")

In [54]:
last_run

,home_run,strikeout,walk,events,name_first,name_last,key_mlbam,run_date,has_been_posted,posted_time
0,2,2,1,5,mike,trout,545361,2022-06-16,False,NaN


In [55]:
last_run.run_date.max()

'2022-06-16'

In [56]:
not_yet_posted = last_run[last_run["has_been_posted"] == False]

if not_yet_posted.shape[0] > 0:
    not_yet_posted.iloc[0]

In [57]:
last_run.loc[(last_run["key_mlbam"] == 664913) & (last_run["run_date"] == "2022-06-12"), "has_been_posted"] = True
#tto_df[(tto_df["key_mlbam"] == key_mlbam) & (tto_df["run_date"] == run_date)]["posted_time"] = "now"

In [58]:
if None:
    pass
else:
    print("none is false")

none is false


In [59]:
last_run

,home_run,strikeout,walk,events,name_first,name_last,key_mlbam,run_date,has_been_posted,posted_time
0,2,2,1,5,mike,trout,545361,2022-06-16,False,NaN


In [60]:
# double check this!

tto_events.loc[tto_events["batter"].isin(player_ids)]

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1305,FF,2022-06-16,92.3,-2.22,4.96,"Festa, Matt",545361,670036,strikeout,swinging_strike,...,1,4,1,4,1,Infield shift,Standard,197,0.007,-0.339
1814,SI,2022-06-16,85.8,-2.84,5.09,"Romo, Sergio",545361,489265,home_run,hit_into_play,...,0,4,0,4,0,Infield shift,Standard,245,-0.132,1.889
226,FF,2022-06-16,96.8,-1.36,5.85,"Kirby, George",545361,669923,strikeout,swinging_strike,...,0,2,0,2,0,Infield shift,Standard,220,0.013,-0.134
784,FF,2022-06-16,95.3,-1.61,5.82,"Kirby, George",545361,669923,home_run,hit_into_play,...,0,2,0,2,0,Standard,Standard,218,-0.219,1.848
1921,FF,2022-06-16,97.2,-1.44,5.9,"Kirby, George",545361,669923,walk,ball,...,0,0,0,0,0,Infield shift,Standard,220,-0.025,0.163


In [99]:
# ok 

sub_data = tto_events.loc[tto_events["batter"] == 545361]
first_test_event = sub_data.iloc[0]

for x in sorted(sub_data.at_bat_number):
    print(x)

2
20
36
52
69


In [100]:
sub_data.hit_distance_sc

1305    <NA>
1814     411
226     <NA>
784      344
1921    <NA>
Name: hit_distance_sc, dtype: Int64

In [101]:
first_test_event.batter

545361

In [102]:
# want to come up with a better description

first_test_event



pitch_type                                FF
game_date                2022-06-16 00:00:00
release_speed                           92.3
release_pos_x                          -2.22
release_pos_z                           4.96
                                ...         
if_fielding_alignment          Infield shift
of_fielding_alignment               Standard
spin_axis                                197
delta_home_win_exp                     0.007
delta_run_exp                         -0.339
Name: 1305, Length: 92, dtype: object

In [103]:
pitch_abbreviation_to_name = {"FF" : "4-Seam Fastball",
                              "CU" : "Changeup",
                              "FC" : "Cutter",
                              "EP" : "Eephus",
                              "FO" : "Forkball",
                              "KN" : "Knuckleball",
                              "KC" : "Knuckle-curve",
                              "SC" : "Screwball",
                              "SI" : "Sinker",
                              "SL" : "Slider",
                              "FS" : "Splitter",
                              "FT" : "2-Seam Fastball"
                             }


In [105]:
description = ""

if first_test_event["events"] == "home_run":
    description += "HR; " + str(first_test_event["hit_distance_sc"]) \
        + " ft, " + str(first_test_event["launch_speed"]) + " mph, " + str(first_test_event["launch_angle"]) + "° launch"
    
    description += "\n"
    
else:
    event = "K" if first_test_event["events"] == "strikeout" else "BB"
    
    try:
        readable_pitch_name = pitch_abbreviation_to_name[first_test_event["pitch_type"]]
    except KeyError as e:
        readable_pitch_name = ""
    
    description += event + "; " + readable_pitch_name + ", " + str(first_test_event["release_speed"]) + " mph"
    description += "\n"

print(description)


K; 4-Seam Fastball, 92.3 mph



In [32]:
first_test_event["events"]

'walk'

In [110]:
str(first_test_event["game_date"].date())


'2022-06-16'

In [33]:
game_pk = tto_events.iloc[0].game_pk
game_pk

663397

### I want to try to implement some way to get the link to the video clip

In [28]:
first_test_event

pitch_type                                ST
game_date                2022-06-16 00:00:00
release_speed                           82.7
release_pos_x                           1.84
release_pos_z                           4.94
                                ...         
if_fielding_alignment               Standard
of_fielding_alignment               Standard
spin_axis                                304
delta_home_win_exp                     0.086
delta_run_exp                          0.472
Name: 1586, Length: 92, dtype: object

In [35]:
import requests as re

bs_endpoint = "https://baseballsavant.mlb.com/gf?game_pk=" 


resp = re.get(bs_endpoint + str(game_pk))

In [38]:
resp_json = resp.json()

In [61]:
# so at_bat_number is for the whole game, so I can concat the two part of that field to get the whole thing.
test_df = pd.DataFrame(resp_json["team_away"])

test_df = pd.concat([test_df, pd.DataFrame(resp_json["team_home"])])

print(test_df.head())

this_ab = test_df[test_df["ab_number"] == first_test_event.at_bat_number]

last_pitch = this_ab[this_ab["pitch_number"] == this_ab["pitch_number"].max()]
last_pitch

                                play_id  inning  ab_number  cap_index  outs  \
0  bd01d8ed-b360-4a6a-8955-cbf0b95a29df       1          5         34     0   
1  80491dd1-2ac0-474e-bff0-ff824549ae27       1          5         34     0   
2  d3125eb0-ab51-430e-93b2-653751b221bf       1          5         34     0   
3  e2c908cc-b9b3-4e42-a5b4-08eb4da419ec       1          5         34     0   
4  00580572-f50e-4b2e-b982-51cbca3c91f5       1          5         34     0   

   batter stand    batter_name  pitcher p_throws  ...   xba hit_angle  \
0  669016     L  Brandon Marsh   592836        R  ...   NaN       NaN   
1  669016     L  Brandon Marsh   592836        R  ...   NaN       NaN   
2  669016     L  Brandon Marsh   592836        R  ...   NaN       NaN   
3  669016     L  Brandon Marsh   592836        R  ...   NaN       NaN   
4  669016     L  Brandon Marsh   592836        R  ...  .890        11   

  is_barrel    hc_x     hc_x_ft    hc_y     hc_y_ft runnerOn2B  runnerOn1B  \
0       

,play_id,inning,ab_number,cap_index,outs,batter,stand,batter_name,pitcher,p_throws,...,xba,hit_angle,is_barrel,hc_x,hc_x_ft,hc_y,hc_y_ft,runnerOn2B,runnerOn1B,runnerOn3B
146,883f8743-2dd6-4741-a6f0-229ab20fb6a8,9,76,14,3,435559,R,Kurt Suzuki,621242,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:

base_video_url = "https://baseballsavant.mlb.com/sporty-videos?playId="

vid_url = base_video_url + str(last_pitch.play_id.iloc[0])

In [56]:
vid_url

'https://baseballsavant.mlb.com/sporty-videos?playId=883f8743-2dd6-4741-a6f0-229ab20fb6a8'

In [84]:
import requests as re


# turn it into a function now!


def get_video_clip_urls(actual_row, tto_events):
    """
    prepares the information needed for a post
    
    Args:
        actual_row: (DataFrame)
            something here
    
    Modifies:
    
    Returns:
        dict with title, alt fields
    """
    # this is the thing we will return
    list_vid_urls = []
    
    # filter to just the batter we care about
    sub_data = tto_events.loc[tto_events["batter"] == actual_row.key_mlbam]
    
    print(sub_data.game_pk.iloc[0])
    
    bs_game_data_endpoint = "https://baseballsavant.mlb.com/gf?game_pk=" 

    resp = re.get(bs_game_data_endpoint + str(sub_data.game_pk))
    
    if resp.status_code != 200:
        return []
        
    resp_json = resp.json()
    
    # this means we have data, but need to find the play ids
    # so at_bat_number is for the whole game, so I can concat the two part of that field to get the whole thing.
    game_data_df = pd.DataFrame(resp_json["team_away"])
    game_data_df = pd.concat([game_data_df, pd.DataFrame(resp_json["team_home"])])



    bs_video_endpoint = "https://baseballsavant.mlb.com/sporty-videos?playId="
    
    for this_at_bat_number in sorted(sub_data.at_bat_number):
        
        # get the data for the whole AB
        this_ab = game_data_df[game_data_df["ab_number"] == this_at_bat_number]

        # the last pitch is what we care about
        last_pitch = this_ab[this_ab["pitch_number"] == this_ab["pitch_number"].max()]
        
        vid_url = bs_video_endpoint + str(last_pitch.play_id.iloc[0])
        
        list_vid_urls.append(vid_url)
        
    return list_vid_urls
    
    

In [82]:
last_run.iloc[0]

home_run                    1
strikeout                   2
walk                        1
events                      4
name_first               seth
name_last               brown
key_mlbam              664913
run_date           2022-06-12
has_been_posted          True
posted_time               NaN
Name: 0, dtype: object

In [87]:
sample_output = ['https://baseballsavant.mlb.com/sporty-videos?playId=b8c48083-e84c-43db-9195-abf4e01b0dc5',
                 'https://baseballsavant.mlb.com/sporty-videos?playId=3b1b9b78-4018-4729-b466-928eebc5e6e7',
                 'https://baseballsavant.mlb.com/sporty-videos?playId=0200153c-a296-499d-8b1f-c5adeb8624fe',
                 'https://baseballsavant.mlb.com/sporty-videos?playId=17824e01-305f-4804-9f63-649ab8e34071']


In [89]:
text = "Shout out to the three true outcome king of the day \n\n"

text += "See the clips of these events here:\n"
for url in sample_output:
    
    text += url + "\n"

print(text)

Shout out to the three true outcome king of the day 

See the clips of the play here:
https://baseballsavant.mlb.com/sporty-videos?playId=b8c48083-e84c-43db-9195-abf4e01b0dc5
https://baseballsavant.mlb.com/sporty-videos?playId=3b1b9b78-4018-4729-b466-928eebc5e6e7
https://baseballsavant.mlb.com/sporty-videos?playId=0200153c-a296-499d-8b1f-c5adeb8624fe
https://baseballsavant.mlb.com/sporty-videos?playId=17824e01-305f-4804-9f63-649ab8e34071



In [ ]:
# this is stupid but the plotting is based on team name, not abbreviation

# TODO: double check this! 
abbreviation_to_name = {
    "LAA" : "angels",
    "HOU" : "astros",
    "OAK" : "athletics",
    "TOR" : "blue_jays",
    "ATL" : "braves",
    "MIL" : "brewers",
    "STL" : "cardinals",
    "CHC" : "cubs",
    "AZ" : "diamondbacks",
    "LAD" : "dodgers",
    "SF" : "giants",
    "CLE" : "indians",
    "SEA" : "mariners",
    "MIA" : "marlins",
    "NYM" : "mets",
    "WSH" : "nationals",
    "BAL" : "orioles",
    "SD" : "padres",
    "PHI" : "phillies",
    "PIT" : "pirates",
    "TEX" : "rangers",
    "TB" : "rays",
    "BOS" : "red_sox",
    "CIN" : "reds",
    "COL" : "rockies",
    "KC" : "royals",
    "DET" : "tigers",
    "MIN" : "twins",
    "CWS" : "white_sox",
    "NYY" : "yankees",
    "else" : "generic"
}


In [ ]:
# check if there are none!

for row in only_three_outcomes.iterrows():
    actual_row = row[1]
    
    player_name = actual_row.name_first.capitalize() + " " + actual_row.name_last.capitalize()
        
    sub_data = tto_events.loc[tto_events["batter"] == actual_row.key_mlbam]
    
    # TODO: put all of the things in the title
    title = player_name + " " + "HR(s): " + str(actual_row.home_run) +\
    ", " + "BB(s): " + str(actual_row.walk) + ", and K(s): " + str(actual_row.strikeout)
    
    # Abbreviation
    abbv = sub_data.home_team.iloc[0]
    
    team_nickname = ""
    try:
        team_nickname = abbreviation_to_name[abbv]
    except KeyError:
        team_nickname = abbreviation_to_name["else"]
    
    # Make alt text
    alt="A figure that includes an outline of the {0} stadium with home runs marked. The title reads {1}".format(abbv, title)
    
    # figure out how to save this as png
    ax = spraychart(sub_data, team_nickname, title = title, height = 400, width=400)
        
    ax.figure.savefig("data/image.png", metadata = {"alt" : alt})
    plt.close(fig)    # close the figure window
    


In [ ]:
# this is for gut checks

id_to_check = 669242

tto_events.loc[tto_events["batter"] == id_to_check]

In [ ]:
total_counts[id_to_check]

In [20]:
df[df["batter"] == id_to_check].game_pk

2764    663179
2796    663179
2898    663179
2951    663179
3119    663179
3133    663179
602     663179
1628    663179
1647    663179
1679    663179
1733    663179
4579    663179
1803    663152
1854    663152
1909    663152
1947    663152
1823    663152
1856    663152
1898    663152
1945    663152
1978    663152
2031    663152
311     663152
331     663152
361     663152
376     663152
1821    663152
1838    663152
1895    663152
1935    663152
4341    663152
4467    663152
4661    663152
Name: game_pk, dtype: Int64

In [60]:
from mastodon import Mastodon

#   Set up Mastodon

# I think I also need to use os for an access token directly
mastodon = Mastodon(
    access_token = 'token.secret',
    api_base_url = 'https://mast.mastodonsports.social/'
)

media = mastodon.media_post("data/image.png", description=alt)
#mastodon.status_post("Shout out to the three true outcome king of the day", media_ids=media)

MastodonNetworkError: Could not complete request: HTTPSConnectionPool(host='mast.mastodonsports.socialdfg', port=443): Max retries exceeded with url: /api/v1/media (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026A50072D70>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))